# Leren: Programming assignment 4

**Student 1:**  <span style="color:red">Wim Berkelmans</span> (<span style="color:red">10793674</span>)<br>
**Student 2:** <span style="color:red">Philip Bouman</span> (<span style="color:red">10668667</span>)<br>

-----------------------------------

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as linalg
import scipy.ndimage
import operator
import math
import sklearn
from sklearn import linear_model, datasets
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors

# Load data from .csv 
def readData():
    training_set = np.loadtxt('digits123-1.csv', skiprows = 1, delimiter=';');
    test_set = np.loadtxt('digits123-2.csv', skiprows = 1, delimiter=';');
    return training_set, test_set

## 1. K-nearest neighbour
####  a) Implementation

In [7]:
# Calculate Euclidian distance between two values
def euclideanDistance(data1, data2):
    dist = 0
    for i in range(len(data1)-1):
        dist += pow((data1[i] - data2[i]), 2)
        
    return math.sqrt(dist)
    
# Get most similar neighbors
def getNeighbors(training_set, test_set, k):
    distances = []
    for i in range(len(training_set)):
        dist = euclideanDistance(training_set[i], test_set)
        distances.append((training_set[i], dist))
        
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i][0])
        
    return neighbors

# Get votes for response
def getResponse(neighbors):
    classVotes = {}
    for i in range(len(neighbors)):
        response = neighbors[i][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
        sortedVotes = sorted(classVotes.iteritems(), key=operator.itemgetter(1), reverse=True)

    return sortedVotes[0][0]
            
# Count number of correct predictions
def getAccuracy(test_set, target):
    correct = 0
    for i in range(len(test_set)):
        if test_set[i][-1] == target[i]:
            correct += 1
            
    return (correct/float(len(test_set))) * 100.0
    
# Main method
def k_nearest_neighbor(k):
    training_set, test_set = readData()
    predictions = []
    
    for i in range(len(test_set)):
        neighbors = getNeighbors(training_set, test_set[i], k)
        result = getResponse(neighbors)
        predictions.append(result)
        
    accuracy = getAccuracy(test_set, predictions)
    print "Accuracy: ", accuracy 
    
# run with differnt k's
k_nearest_neighbor(1)
k_nearest_neighbor(3)
k_nearest_neighbor(5)

Accuracy:  98.7447698745
Accuracy:  97.9079497908
Accuracy:  97.9079497908


#### b) Weighted nearest neighbour

#### c) Predictive value

## 2. Compare classifiers
#### a) Implementation

In [3]:
# load data from file
def readDigits(selection):
    if selection == 'train':
        data = np.loadtxt('digits123-1.csv', delimiter=';')
    elif selection == 'test':
        data = np.loadtxt('digits123-2.csv', delimiter=';')
        
    return data

def initTrainTest():
    DigitsTrain = readDigits('train')
    TrainY = DigitsTrain[:,-1] # Y target values, last column of Digits
    Train = np.delete(DigitsTrain, -1, 1) # remove target values
    DigitsTest = readDigits('test')
    TestY = DigitsTest[:,-1] # Y target values, last column of Digits
    Test = np.delete(DigitsTest, -1, 1) # remove target values
    return Train, TrainY, Test, TestY

**Logistic Regression**



In [4]:
def logReg():
    Train, TrainY, Test, TestY = initTrainTest()
    logreg = linear_model.LogisticRegression(C=0.001, solver='newton-cg', 
                                    max_iter=100, multi_class='multinomial')
    logreg.fit(Train, TrainY)

    return logreg.score(Test,TestY)

logReg()

0.95416666666666672

**Neural Network**



In [5]:
def neuralNetwork():
    Train, TrainY, Test, TestY = initTrainTest()
    logreg = MLPClassifier(solver='adam', activation='logistic', 
                           alpha=0.001, hidden_layer_sizes=(50,))
    logreg.fit(Train, TrainY)

    return logreg.score(Test,TestY)

neuralNetwork()

0.95416666666666672

**Nearest Neighbour**



In [6]:
def nearestN(k):
    Train, TrainY, Test, TestY = initTrainTest()
    logreg = neighbors.KNeighborsClassifier(k, weights='distance')
    logreg.fit(Train, TrainY)

    return logreg.score(Test,TestY)

print "NNB k=1 ", nearestN(1)
print "NNB k=3 ", nearestN(3)
print "NNB k=5 ", nearestN(5)

NNB k=1  0.9875
NNB k=3  0.983333333333
NNB k=5  0.983333333333
